In [ ]:
from pathlib import Path
import random
import re
from typing import Literal

from datasets import Dataset
import optuna
import torch
from torch.utils.data import DataLoader
from transformers import (
  AutoConfig,
  DataCollatorForLanguageModeling,
  GPT2LMHeadModel, GPT2Tokenizer,
  Trainer, TrainerCallback, TrainerControl,
  TrainerState, TrainingArguments,
)

# Demonstration of GPT-2 architecture model pretraining for C++ code completion

## Introduction
In this notebook is demonstrated how a [GPT-2](#GPT-2) architecture model can be pretrained for C++ code completion, based on a single source file. The goal is to limit the required computing resources for training and evaluation. The demonstrated method utilizes the [Hugging Face Transformers](#Hugging-Face-Transformers) and [PyTorch](#PyTorch) deep learning libraries, and more specifically - the [DistilGPT2](#DistilGPT2) language model without pretrained weights.

## Configuration

In [ ]:
# General
RANDOM_SEED = 42

# Data
CPP_FILE = "engine.cc"
DATA_DIR = "data"
SAMPLE_SPAN = 3
SAMPLE_STRIDE = 1

# Optuna
OPTUNA_ENABLED = False
OPTUNA_EPOCHS = 2

# Training
BATCH_SIZE = 4
LEARN_RATE = 1e-4
LEARN_RATE_SCHEDULER = "inverse_sqrt"
MODEL_NAME = "distilbert/distilgpt2"
MODELS_DIR = "models"
TRAIN_EPOCHS = 5
VALIDATION_SPLIT = 0.2
WARMUP_STEPS = 0
WEIGHT_DECAY = 0.05

# Text generation
MIN_GEN_PROB = 0.95
MAX_GEN_TOKENS = 100

## Dataset retrieval

For simplicity reasons, for dataset is selected a C++ source file `CPP_FILE` located at `DATA_DIR`. Hence, the dataset size depends on the size of the file.

In [ ]:
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
raw_text = (Path() / DATA_DIR / CPP_FILE).read_text()

## Tokenizer

The [DistilGPT2](#DistilGPT2) model uses the [GPT-2](#GPT-2) tokenizer, which in turn uses [Byte pair encoding](#Byte-pair-encoding). Thus, spaces, new lines and braces, which are important not just for C++ but also for many other programming languages, are part of the vocabulary and their embeddings can be trained and fine tuned.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = tokenizer.eos_token_id
inverted_vocab = {v: k for k, v in tokenizer.get_vocab().items()}

## Dataset preprocessing

First, single-line comments, *#include*s and empty lines are removed.

Then, based on the `SAMPLE_SPAN` and `SAMPLE_STRIDE` settings, the selected C++ source file is split into lines which are combined into potentially overlapping groups of length `SAMPLE_SPAN` and offset `SAMPLE_STRIDE`.

The formed groups are then shuffled and split into training and validation sets, based on the `VALIDATION_SPLIT` setting.

In [ ]:
raw_lines = [re.sub(r"([^\s]*)\s*//.*", r"\1", line)
             for line in raw_text.splitlines() if line]
raw_lines = [line
             for line in raw_lines
             if line and not line.lstrip().startswith("#include")]

samples = ["\n".join(raw_lines[i:i+SAMPLE_SPAN])
           for i in range(0, len(raw_lines)-SAMPLE_SPAN+1, SAMPLE_STRIDE)]
random.shuffle(samples)

val_idx = int(VALIDATION_SPLIT * len(samples))
train_samples = samples[:-val_idx]
train_ds = Dataset.from_dict(tokenizer(train_samples))
val_samples = samples[-val_idx:]
val_ds = Dataset.from_dict(tokenizer(val_samples))

## Preparation for training

Standard [Hugging Face Transformers](#Hugging-Face-Transformers) API for [PyTorch](#PyTorch), such as the `Trainer`, `TrainingArguments` and `TrainerCallback` classes, is utilized to configure the [DistilGPT2](#DistilGPT2) model for training.

If the `OPTUNA_ENABLED` setting is *True*, then, instead of a training process, starts a hyperparameter tuning process via [Optuna](#Optuna). Thus, settings such as learning rate value and schedule, batch size and weight decay can be tuned.

Additionally, a custom `EvaluationCallback` (based on `TrainerCallback`) is implemented to provide, via the *evaluate_model* function, for evaluation of the model's text generation capabilities on:
- a single, randomly selected, C++ code sample per training epoch;
- a custom C++ fragment during inference.

The *evaluate_model* function displays various evaluation data such as:
- the C++ code fragment;
- the tokens' representation as per the tokenizer;
- the prompt as per the *prompt_strategy* parameter, i.e., whether the first half of *cpp_text*, the second half or the whole *cpp_text* is to be used as prompt for code completion;
- the generated code without the prompt, as per the *suggest_changes* function.

The generated code is displayed only up to `MAX_GEN_TOKENS` and up to the part for which the model is confident, as per the `MIN_GEN_PROB` setting. Thus, during the initial training epochs there may be no generated code.

In [ ]:
def evaluate_model(
  model: GPT2LMHeadModel,
  tokenizer: GPT2Tokenizer,
  dataset: Dataset | None = None,
  cpp_text: str | None = None,
  prompt_strategy: Literal["start", "end", "all"] = "all",
) -> None:
  if dataset is None and cpp_text is None:
    raise ValueError("evaluate_model: "
                     "one of dataset and cpp_text must be set.")

  model.eval()

  if dataset is not None:
    index = random.randint(0, len(dataset)-1)
    ids = dataset["input_ids"][index]
    cpp_text = tokenizer.decode(ids)
  else:
    ids = tokenizer.encode(cpp_text)  # type: ignore
  print("-" * 80,
        f"[ C++ CODE ]\n\n{cpp_text}",
        sep="\n")

  tokens_repr = " ".join([inverted_vocab[i] for i in ids])
  print("-" * 80,
        f"[ TOKENS ]\n\n{tokens_repr}",
        sep="\n")

  if prompt_strategy == "start":
    prompt_ids = ids[:len(ids)//2]
  elif prompt_strategy == "end":
    prompt_ids = ids[len(ids)//2:]
  else:
    prompt_ids = ids
  prompt = tokenizer.decode(prompt_ids)
  print("-" * 80,
        f"[ PROMPT ]\n\n{prompt}",
        sep="\n")

  changes = suggest_changes(model, tokenizer,
                            device_type=model.device.type,
                            min_p=MIN_GEN_PROB, prompt=prompt)
  print("-" * 80,
        f"[ GENERATED ]\n\n{changes}",
        "-" * 80,
        sep="\n")


def suggest_changes(
  model: GPT2LMHeadModel,
  tokenizer: GPT2Tokenizer,
  device_type: str,
  prompt: str,
  min_p: float = MIN_GEN_PROB,
) -> str:
  if not prompt:
    raise ValueError("suggest_changes: prompt is empty.")

  ids: list[int] = tokenizer(prompt)["input_ids"]  # type: ignore
  input_len = len(ids)
  while len(ids) - input_len < MAX_GEN_TOKENS:
    logits = model(torch.tensor(ids).to(model.device)).logits
    predictions = torch.softmax(logits[-1], dim=-1)
    next_id = int(torch.argmax(predictions).item())
    if predictions[next_id] < min_p:
      break
    ids += [next_id]
  return tokenizer.decode(ids[input_len:])


training_args = TrainingArguments(
  output_dir=MODELS_DIR,
  eval_strategy="epoch",
  per_device_train_batch_size=BATCH_SIZE,
  per_device_eval_batch_size=BATCH_SIZE,
  learning_rate=LEARN_RATE,
  lr_scheduler_type=LEARN_RATE_SCHEDULER,
  warmup_steps=WARMUP_STEPS,
  metric_for_best_model="eval_loss" if OPTUNA_ENABLED else None,
  save_strategy="epoch",
  save_total_limit=1,
  save_only_model=False,
  seed=RANDOM_SEED,
  dataloader_drop_last=True,
  load_best_model_at_end=False,
  weight_decay=WEIGHT_DECAY,
  report_to=["tensorboard"],
  push_to_hub=False,
  num_train_epochs=OPTUNA_EPOCHS if OPTUNA_ENABLED else TRAIN_EPOCHS,
)


if not OPTUNA_ENABLED:
  class EvaluationCallback(TrainerCallback):
    def on_epoch_end(
      self,
      args: TrainingArguments,
      state: TrainerState,
      control: TrainerControl,
      model: GPT2LMHeadModel | None = None,
      processing_class: GPT2Tokenizer | None = None,
      train_dataloader: DataLoader | None = None,
      **kwargs,
    ) -> None:
      if (model is not None
          and processing_class is not None
          and train_dataloader is not None):
        evaluate_model(model, processing_class,
                       dataset=train_dataloader.dataset)  # type: ignore


def model_init(trial: optuna.Trial) -> GPT2LMHeadModel:
  try:
    return GPT2LMHeadModel.from_pretrained(MODELS_DIR,
                                           device_map="auto",
                                           torch_dtype="auto")
  except Exception as e:
    print("model_init:", e)
    print("A model without pretrained weights to be loaded...")
    return GPT2LMHeadModel(AutoConfig.from_pretrained(MODEL_NAME))


trainer = Trainer(
  model_init=model_init,  # type: ignore
  processing_class=tokenizer,
  args=training_args,
  train_dataset=train_ds,
  eval_dataset=val_ds,
  data_collator=DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
  ),
  callbacks=None if OPTUNA_ENABLED else [EvaluationCallback()],  # type: ignore
)

## Training and evaluation

If `OPTUNA_ENABLED` is *True*, then in the `optuna_hp_space` function are configured the trial parameters. The optimization process is performed with regard to the validation loss.

If `OPTUNA_ENABLED` is *False*, then a training process starts. The final validation loss and perplexity metrics are reported once the training process completes.

In [ ]:
if OPTUNA_ENABLED:
  def optuna_hp_space(trial: optuna.Trial) -> dict[str, str | float]:
    return {
      "learning_rate": trial.suggest_float(
        "learning_rate", 1e-6, 1e-4, step=1e-6
      ),
      "weight_decay": trial.suggest_float(
        "weight_decay", 0, 1e-1, step=1e-2
      ),
      "lr_scheduler_type": trial.suggest_categorical(
        "lr_scheduler_type", ["constant", "cosine",
                              "inverse_sqrt", "linear",
                              "reduce_lr_on_plateau"]
      ),
      "per_device_train_batch_size": trial.suggest_categorical(
        "per_device_train_batch_size", [4]
      ),
    }
  optuna_sampler = optuna.samplers.TPESampler(seed=RANDOM_SEED)
  best_trials = trainer.hyperparameter_search(
    hp_space=optuna_hp_space,  # type: ignore
    direction="minimize",
    backend="optuna",
    n_trials=20,
    sampler=optuna_sampler,
  )
  print(best_trials)
else:
  trainer.train()
  eval_results = trainer.evaluate()
  print(eval_results)
  print("Perplexity: "
        f"{torch.exp(torch.tensor(eval_results['eval_loss'])).item():.2f}")
  trainer.save_model()

## Testing

In [ ]:
if not OPTUNA_ENABLED:
  evaluate_model(trainer.model, tokenizer,  # type: ignore
                 cpp_text=samples[0])

## References

<br><br>

### APA style for references
American Psychological Association. (2022). Creating an APA Style reference list guide. https://apastyle.apa.org/instructional-aids/creating-reference-list.pdf

American Psychological Association. (2024). APA Style common reference examples guide. https://apastyle.apa.org/instructional-aids/reference-examples.pdf

<br><br>

### Tokenization
#### Byte-pair encoding
Sennrich, R., Haddow, B., & Birch, A. (2015). Neural machine translation of rare words with subword units. arXiv preprint arXiv:1508.07909. https://arxiv.org/abs/1508.07909
- [Byte pair encoding - Wikipedia](https://en.wikipedia.org/wiki/Byte_pair_encoding)

<br><br>

### Machine learning models
#### GPT-2
Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). Language models are unsupervised multitask learners. OpenAI blog, 1(8), 9. https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf

#### DistilGPT2
Sanh, V., Debut, L., Chaumond, J., & Wolf, T. (2019). DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter. arXiv preprint arXiv:1910.01108. https://arxiv.org/abs/1910.01108

<br><br>

### Guides and tutorials
- [distilbert/distilgpt2 · Hugging Face](https://huggingface.co/distilbert/distilgpt2)
- [Hugging Face - Documentation](https://huggingface.co/docs)

<br><br>

### Libraries
#### Hugging Face Transformers
Wolf, T., Debut, L., Sanh, V., Chaumond, J., Delangue, C., Moi, A., Cistac, P., Ma, C., Jernite, Y., Plu, J., Xu, C., Le Scao, T., Gugger, S., Drame, M., Lhoest, Q., & Rush, A. M. (2020). Transformers: State-of-the-Art Natural Language Processing [Conference paper]. 38–45. https://www.aclweb.org/anthology/2020.emnlp-demos.6
- [Transformers](https://huggingface.co/docs/transformers/index)

#### PyTorch
Ansel, J., Yang, E., He, H., Gimelshein, N., Jain, A., Voznesensky, M., Bao, B., Bell, P., Berard, D., Burovski, E., Chauhan, G., Chourdia, A., Constable, W., Desmaison, A., DeVito, Z., Ellison, E., Feng, W., Gong, J., Gschwind, M., Hirsh, B., Huang, S., Kalambarkar, K., Kirsch, L., Lazos, M., Lezcano, M., Liang, Y., Liang, J., Lu, Y., Luk, C., Maher, B., Pan, Y., Puhrsch, C., Reso, M., Saroufim, M., Siraichi, M. Y., Suk, H., Suo, M., Tillet, P., Wang, E., Wang, X., Wen, W., Zhang, S., Zhao, X., Zhou, K., Zou, R., Mathews, A., Chanan, G., Wu, P., & Chintala, S. (2024). PyTorch 2: Faster Machine Learning Through Dynamic Python Bytecode Transformation and Graph Compilation [Conference paper]. 29th ACM International Conference on Architectural Support for Programming Languages and Operating Systems, Volume 2 (ASPLOS '24). https://doi.org/10.1145/3620665.3640366
- [Start Locally | PyTorch](https://pytorch.org/get-started/locally)

<br><br>

### Tools
#### Optuna
Akiba, T., Sano, S., Yanase, T., Ohta, T., & Koyama, M. (2019). Optuna: A next-generation hyperparameter optimization framework [Conference paper]. *Proceedings of the 25th ACM SIGKDD international conference on knowledge discovery & data mining*, 2623–2631. https://doi.org/10.1145/3292500.3330701
- [Optuna: A hyperparameter optimization framework](https://optuna.readthedocs.io/en/stable/)